In [1]:
# Importamos las bibliotecas necesarias y el dataset previamente limpiado

import pandas as pd
import numpy as np

from sklearn.metrics import average_precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import StratifiedKFold

from itertools import combinations
import sklearn.metrics as metrics

In [2]:
data_gym = pd.read_excel('DatosParaModelo.xlsx')

In [3]:
# Se eliminan los registros del último mes para aquellos IDs que sólo tienen un registro

# Se obtienen los IDs que sólo tienen 1 registro y los IDs presentes en el último mes
Conteo_registros = data_gym['IdPersona'].value_counts()
ID_multiples_registros = Conteo_registros[Conteo_registros == 1].index
ID_ultimo_mes = data_gym[data_gym['Mes'] == '2023-12-01']['IdPersona'].tolist()

# Se genera una lista con los IDs comunes en ambas listas y se descartan del DataFrame
IDs_eliminar = set(ID_multiples_registros).intersection(ID_ultimo_mes)

data_gym_sin_ultimos_registros = data_gym[~data_gym['IdPersona'].isin(IDs_eliminar)]

# Logistic Regression

In [4]:
def Logistic_Regression (DataFrame, Splits):

    lista_AUC_kfolds = []

    Modelo_reg_log = LogisticRegression(penalty=None, max_iter=3000)

    # Se configuran las variables de entrada y salida, la lista de identificadores de clientes y se prepara las divisiones del dataset con estratificación
    X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
    y = DataFrame['Baja']
    Id_Clientes = DataFrame['IdPersona']
    kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)

    # Bucle para iterar sobre los diferentes folds separando entre train y test con divisiones estratificadas según Ids de clientes
    for Train_data, Test_data in kfold.split(X, y, groups = Id_Clientes):
        X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
        y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]

        # Entrenamiento del modelo y cálculo del AUC_PR sobre el conjunto de test
        Modelo_reg_log.fit(X_train, y_train)
        Predicciones_reg = Modelo_reg_log.predict_proba(X_test)
        AUC_PR = average_precision_score(y_test, Predicciones_reg[:, 1])
        lista_AUC_kfolds.append(AUC_PR)

    # Resultado
    Mean_AUC = np.mean(lista_AUC_kfolds)
    return Mean_AUC

# Random Forest

In [5]:
def Random_Forest(Profundidades, Estimadores, DataFrame, Splits):

  lista_parametros = []
  lista_AUC = []

  for Depth in Profundidades:
    for n_estimators in Estimadores:
      lista_AUC_kfolds = []
      
      # Se configuran las variables de entrada y salida, la lista de identificadores de clientes y se prepara las divisiones del dataset con estratificación
      X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
      y = DataFrame['Baja']
      Id_Clientes = DataFrame['IdPersona']
      kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)

      # Bucle para iterar sobre los diferentes folds separando entre train y test con divisiones estratificadas según Ids de clientes   
      for Train_data, Test_data in kfold.split(X, y, Id_Clientes):
        X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
        y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]    
    
        # Entrenamiento del modelo y cálculo del AUC_PR sobre el conjunto de test
        Modelo_random_forest = RandomForestClassifier(max_depth=Depth, n_estimators=n_estimators, random_state=1).fit(X_train, y_train)
        Predicciones_rndm_frst = Modelo_random_forest.predict_proba(X_test)
        AUC_PR = average_precision_score(y_test, Predicciones_rndm_frst[:, 1])
        lista_AUC_kfolds.append(AUC_PR)

      # Promediado de los resultados de los diferentes folds y agregado de resultado en las listas
      Mean_AUC = np.mean(lista_AUC_kfolds)
      lista_AUC.append(Mean_AUC)    
      lista_parametros.append([Depth, n_estimators])

  # Resultados
  AUC_max_value = np.argmax(lista_AUC)
  return (lista_AUC[AUC_max_value], lista_parametros[AUC_max_value][0], lista_parametros[AUC_max_value][1])

# Gradient Boosting Classifier

In [6]:
def Gradient_Boosting (Profundidades, Estimadores, DataFrame, Splits):

    lista_parametros = []
    lista_AUC = []

    for Depth in Profundidades:
        for n_estimators in Estimadores:
            lista_AUC_kfolds = []

            # Se configuran las variables de entrada y salida, la lista de identificadores de clientes y se prepara las divisiones del dataset con estratificación
            X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
            y = DataFrame['Baja']
            Id_Clientes = DataFrame['IdPersona']
            kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)
            
            # Bucle para iterar sobre los diferentes folds separando entre train y test con divisiones estratificadas según Ids de clientes
            for Train_data, Test_data in kfold.split(X, y, groups = Id_Clientes):
                X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
                y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]

                # Entrenamiento del modelo y cálculo del AUC_PR sobre el conjunto de test
                Modelo_xgboost = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=Depth, objective='binary:logistic').fit(X_train, y_train)
                Predicciones_xgb = Modelo_xgboost.predict_proba(X_test)
                AUC_PR = average_precision_score(y_test, Predicciones_xgb[:, 1])
                lista_AUC_kfolds.append(AUC_PR)

            # Promediado de los resultados de los diferentes folds y agregado de resultado en las listas
            Mean_AUC = np.mean(lista_AUC_kfolds)
            lista_AUC.append(Mean_AUC)
            lista_parametros.append([Depth, n_estimators])

    # Resultados
    AUC_max_value = np.argmax(lista_AUC)
    return (lista_AUC[AUC_max_value], lista_parametros[AUC_max_value][0], lista_parametros[AUC_max_value][1])

# Variables de los Modelos

In [7]:
Splits = 50
Profundidades_xgb = [2, 3, 4, 5, 6, 7, 8, 9, 10]
Estimadores_xgb = [20, 40, 60, 80, 100, 120]
Profundidades_forest = [8, 9, 10, 11, 12, 13, 14, 15]
Estimadores_forest = [100, 150, 200, 250, 300]

# Ejecución de Modelos

In [8]:
Result_AUC = Logistic_Regression(data_gym_sin_ultimos_registros, Splits)
print(f'El AUC del Modelo de Regresión Logística es: {Result_AUC}')

El AUC del Modelo de Regresión Logística es: 0.19517948836773905


In [9]:
Result_AUC, Result_profundidad, Result_estimador = Gradient_Boosting(Profundidades_xgb, Estimadores_xgb, data_gym_sin_ultimos_registros, Splits)
print(f'El AUC del modelo XGBoost es {Result_AUC}, para un nivel de profundidad de {Result_profundidad} y consta de {Result_estimador} árboles de decisión')

El AUC del modelo XGBoost es 0.24464388205538598, para un nivel de profundidad de 7 y consta de 20 árboles de decisión


In [10]:
Result_AUC, Result_profundidad, Result_estimador = Random_Forest(Profundidades_forest, Estimadores_forest, data_gym_sin_ultimos_registros, Splits)
print(f'El AUC del modelo Random Forest es {Result_AUC}, para un nivel de profundidad de {Result_profundidad} y consta de {Result_estimador} árboles de decisión')

El AUC del modelo Random Forest es 0.2543031441350968, para un nivel de profundidad de 15 y consta de 300 árboles de decisión
